# **importing libs**

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
import PIL
import tensorflow as tf
import zipfile
import glob
from tensorflow import keras
from keras import layers
from keras.models import Sequential

# **importing dataset**

In [2]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [3]:
#downloading dataset
!kaggle datasets download -d devzohaib/dog-emotions-prediction

 99% 377M/379M [00:03<00:00, 123MB/s]
100% 379M/379M [00:03<00:00, 110MB/s]


In [4]:
#unzipping the dataset and deleting the zipped file
zip_path = '/content/dog-emotions-prediction.zip'
dir_path = '/content/uncompressed'

os.makedirs(dir_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(dir_path)

os.remove(zip_path)

In [5]:
#converting the path from str to path extension
import pathlib
data_dir = pathlib.Path(dir_path)
data_dir

PosixPath('/content/uncompressed')

# deleting 3/4 of the database since it crashed every time

In [6]:
#definig the function
def delete_34_files(path):
    files = glob.glob(path + '/*')

    three_quarters = (len(files)*3) // 4

    for i in range(three_quarters):
        os.remove(files[i])

In [7]:
delete_34_files('/content/uncompressed/images/happy')
delete_34_files('/content/uncompressed/images/angry')
delete_34_files('/content/uncompressed/images/relaxed')
delete_34_files('/content/uncompressed/images/sad')

# processing data

In [8]:
dog_emotion_dict = {
    'angry': list(data_dir.glob('images/angry/*')),
    'happy': list(data_dir.glob('images/happy/*')),
    'relaxed': list(data_dir.glob('images/relaxed/*')),
    'sad': list(data_dir.glob('images/sad/*')),
}
print(dog_emotion_dict.items())

dict_items([('angry', [PosixPath('/content/uncompressed/images/angry/5835898310_cc6531f601_b.jpg'), PosixPath('/content/uncompressed/images/angry/3589344238_300f821286_b.jpg'), PosixPath('/content/uncompressed/images/angry/19925356184_d7bfaefef1_b.jpg'), PosixPath('/content/uncompressed/images/angry/6746001651_a74978817a_b.jpg'), PosixPath('/content/uncompressed/images/angry/51352824788_9169985e0b_b.jpg'), PosixPath('/content/uncompressed/images/angry/4257528727_8d994451f4_b.jpg'), PosixPath('/content/uncompressed/images/angry/6821897995_d905ee4493_b.jpg'), PosixPath('/content/uncompressed/images/angry/5974589854_a8254a86ef_b.jpg'), PosixPath('/content/uncompressed/images/angry/6202537429_1c5300db68_b.jpg'), PosixPath('/content/uncompressed/images/angry/2321142514_d158194a83_b.jpg'), PosixPath('/content/uncompressed/images/angry/50923312618_32d6aea33e_b.jpg'), PosixPath('/content/uncompressed/images/angry/4693701632_db07435d8a_b.jpg'), PosixPath('/content/uncompressed/images/angry/5360

In [9]:
emotions_labels_dict = {
    'angry': 0,
    'happy': 1,
    'relaxed': 2,
    'sad': 3,
}

# Resizing images

In [10]:
X, y = [], []

for emotion_name, images in dog_emotion_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(180,180))
        X.append(resized_img)
        y.append(emotions_labels_dict[emotion_name])
        os.remove(str(image))
import shutil
shutil.rmtree(dir_path)
print(len(y))

3981


In [11]:
X = np.array(X)
y = np.array(y)

# Splitting Data

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.25 ,random_state=0)#default splitting 0.25
del X
del y

#Preprocessing: scaling images so it only ranges from 0-1

In [13]:
X_train_scaled = X_train / 255
del X_train
X_test_scaled = X_test / 255
del X_test

# Trainig the model

In [14]:
#training
num_classes = 4

model = Sequential([
  layers.experimental.preprocessing.RandomRotation(0.1),
  layers.experimental.preprocessing.RandomZoom(0.1),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=
                                                                 True),
              metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=20)

Epoch 1/10
94/94 [==============================] - 109s 1s/step - loss: 1.3931 - accuracy: 0.3032
Epoch 2/10
94/94 [==============================] - 109s 1s/step - loss: 1.3428 - accuracy: 0.3307
Epoch 3/10
94/94 [==============================] - 105s 1s/step - loss: 1.3377 - accuracy: 0.3340
Epoch 4/10
94/94 [==============================] - 104s 1s/step - loss: 1.3255 - accuracy: 0.3441
Epoch 5/10
94/94 [==============================] - 106s 1s/step - loss: 1.3066 - accuracy: 0.3973
Epoch 6/10
94/94 [==============================] - 104s 1s/step - loss: 1.3030 - accuracy: 0.3990
Epoch 7/10
94/94 [==============================] - 104s 1s/step - loss: 1.2914 - accuracy: 0.3906
Epoch 8/10
94/94 [==============================] - 104s 1s/step - loss: 1.2780 - accuracy: 0.4040
Epoch 9/10
94/94 [==============================] - 101s 1s/step - loss: 1.2740 - accuracy: 0.4114
Epoch 10/10
94/94 [==============================] - 103s 1s/step - loss: 1.2712 - accuracy: 0.4121


In [15]:
#evaluation
model.evaluate(X_test_scaled,y_test)

32/32 [==============================] - 11s 329ms/step - loss: 1.3494 - accuracy: 0.3795


[1.3494497537612915, 0.379518061876297]